# **Hands-on: molecular data reading**

Author: Tales Miguel Machado Pereira
RA: 140247
Professor: Dr. Marcos Quiles

### **1.Load and unpack the QM9 dataset**

[Data for 133885 GDB-9 molecules](https://springernature.figshare.com/articles/dataset/Data_for_6095_constitutional_isomers_of_C7H10O2/1057646?backTo=%2Fcollections%2FQuantum_chemistry_structures_and_properties_of_134_kilo_molecules%2F978904&file=3195389)

#### **1.1. About the Dataset**

"Molecular structures and properties are publicly available at Figshare (Data Citation 1) in a plain text XYZ-like format described below. Deposited files include the 133, 885 GDB-1 to GDB-9 molecules (dsgdb9nsd.xyz.tar.bz2), the 6,095 constitutional isomers of C7H10O2 (dsC7O2H10nsd.xyz.tar.bz2), the 100 validation molecules (see Table 1) enthalpies of atomization (validation.txt), and atomic reference data (atomref.txt)."

[Ramakrishnan, R., Dral, P., Rupp, M. et al. Quantum chemistry structures and properties of 134 kilo molecules. Sci Data 1, 140022 (2014).](https://www.nature.com/articles/sdata201422#Sec4)

#### **1.2. General structure of QM9 XYZ files**

- Line 1: Number of atoms in the molecule

- Line 2: Extended header with computed molecular properties (16 values):
    1. gdb ID - Database identifier
    2. A - Rotational constant A (GHz)
    3. B - Rotational constant B (GHz)
    4. C - Rotational constant C (GHz)
    5. μ - Dipole moment (Debye)
    6. α - Isotropic polarizability (Bohr³)
    7. εHOMO - Energy of Highest Occupied Molecular Orbital (Hartree)
    8. εLUMO - Energy of Lowest Unoccupied Molecular Orbital (Hartree)
    9. Δε - Gap between HOMO and LUMO (Hartree)
    10. 〈R²〉 - Electronic spatial extent (Bohr²)
    11. zpve - Zero point vibrational energy (Hartree)
    12. U₀ - Internal energy at 0K (Hartree)
    13. U - Internal energy at 298.15K (Hartree)
    14. H - Enthalpy at 298.15K (Hartree)
    15. G - Free energy at 298.15K (Hartree)
    16. Cv - Heat capacity at 298.15K (cal/mol·K)

- Lines 3 to N+2: Atomic coordinates and charges (5 columns):
    1. Element symbol - Chemical element (C, H, N, O, etc.)
    2. x coordinate - Position in Angstroms
    3. y coordinate - Position in Angstroms
    4. z coordinate - Position in Angstroms
    5. Partial charge - Mulliken partial charge (electrons)
    
- Line N+3: Harmonic vibrational frequencies (cm⁻¹) - varies by molecule size

- Line N+4: Mulliken charges by atom type

- Lines N+5 & N+6: InChI strings for chemical identification and verification

In [ ]:
import tarfile
import pandas as pd

def unpack_database(db_path='db/dsgdb9nsd.xyz.tar.bz2', dest_path='db/qm9_unpacked'):
    """
    Unpack 
    """
    # Unpack the tar.bz2 file
    with tarfile.open(db_path, 'r:bz2') as tar:
        tar.extractall(path=dest_path)

    print(f"\nDatabase unpacked to {dest_path}")


# Path to the tar.bz2 file inside the db folder
db_path = 'db/dsgdb9nsd.xyz.tar.bz2'
dest_path = 'db/qm9_unpacked'
unpack_database(db_path, dest_path)

### **2.XYZ File Parser**

In [11]:
import os
import glob
import pandas as pd
from rdkit import Chem
import warnings
warnings.filterwarnings('ignore')

In [17]:
def parse_xyz_file(file_path):
    """Parse QM9 XYZ file and extract molecular properties and SMILES."""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    
    # Parse basic structure
    n_atoms = int(lines[0])
    header_parts = lines[1].split('\t')
    
    # SMILES is on the line after vibrational frequencies
    smiles_line_idx = 2 + n_atoms + 1
    smiles = lines[smiles_line_idx].split('\t')[0]  # Take first SMILES if duplicated
    
    # Extract key molecular properties
    data = {
        'gdb_id': int(header_parts[0].split()[1]),
        'n_atoms': n_atoms,
        'homo_lumo_gap': float(header_parts[8]),
        'smiles': smiles
    }
    
    return data

In [18]:
def validate_and_canonicalize_smiles(smiles):
    """Validate SMILES and generate canonical version."""
    # Validate original SMILES
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None, False
    
    # Generate canonical SMILES
    canonical_smiles = Chem.MolToSmiles(mol, canonical=True)
    
    return canonical_smiles, True

In [19]:
def process_xyz_files(directory_path, max_files=None):
    """Process XYZ files and extract SMILES and properties."""
    xyz_files = sorted(glob.glob(os.path.join(directory_path, "*.xyz")))
    
    if max_files:
        xyz_files = xyz_files[:max_files]
    
    results = []
    
    for file_path in xyz_files:
        # Parse XYZ file (SMILES already included)
        data = parse_xyz_file(file_path)
        
        # Validate and canonicalize SMILES
        canonical_smiles, is_valid = validate_and_canonicalize_smiles(data['smiles'])
        
        # Only keep valid molecules
        if is_valid:
            data['canonical_smiles'] = canonical_smiles
            results.append(data)
    
    return pd.DataFrame(results)

In [20]:
# Process sample of molecules
df = process_xyz_files('db/qm9_unpacked', max_files=50)

# Save results
df.to_csv('qm9_sample_with_smiles.csv', index=False)

print(f"Processed {len(df)} molecules")
print(f"Sample data:")
print(df[['gdb_id', 'n_atoms', 'canonical_smiles', 'homo_lumo_gap']].head())

Processed 50 molecules
Sample data:
   gdb_id  n_atoms canonical_smiles  homo_lumo_gap
0       1        5                C         0.5048
1       2        4                N         0.3399
2       3        3                O         0.3615
3       4        4              C#C         0.3351
4       5        3              C#N         0.3796


In [16]:
# Uncomment to process the full dataset:
# df_full = process_xyz_files('db/qm9_unpacked')
# df_full.to_csv('qm9_complete_dataset.csv', index=False)